### What's the most expensive listing? What else can you tell me about the listing?
~~~~
SELECT 
	name, 
	neighbourhood, 
	price 
FROM 
	sfo_listings 
ORDER BY 
	price DESC 
LIMIT 5;
~~~~
Output: Columns are name, neighborhood and price.
~~~~
"Test apartment"	"Russian Hill"	"9999"
"3 bed & 2 bath in Diamond Heights with Bay view"	"Diamond Heights"	"9270"
"NobHill Penthouse-Top View&Location"	"Nob Hill"	"9000"
"Bright private room in the Parkside."	"Parkside"	"8000"
"Three Bridges Penthouse by ONEFINESTAY"	"Pacific Heights"	"8000"
~~~~
This is a 'test' apartment and probably not real; Allison is the named host; it's in the Russian Hill neighbourhood and has zero reviews.

### What neighborhoods seem to be the most popular?
~~~~
SELECT 
	neighbourhood,
	sum(number_of_reviews) as tot_reviews_by_neighborhood,
	sum(reviews_per_month) as rev_per_month_by_neighborhood
FROM 
	sfo_listings
GROUP BY 
	neighbourhood
ORDER BY 
	tot_reviews_by_neighborhood DESC
--	rev_per_month_by_neighborhood desc;
LIMIT 3;

Output:
"Mission"	"34412"	"1066.21"
"Western Addition"	"23098"	"719.12"
"Bernal Heights"	"20635"	"697.76"
~~~~
The above 3 neighborhoods have the most total reviews, so they seem to be the most popular.  The Mission neighborhood also has the higest number of reviews per month on average.


###  What time of year is the cheapest time to go to your city? What about the busiest?
~~~~

-- extract the month from the calender_date in order to group by month later.
WITH
    get_month
AS (
	SELECT 
		listing_id,
		price,
		EXTRACT (month from calender_date),
		REPLACE(price, ',', '') AS new_price
	FROM 
 		sfo_calender
	WHERE 
 		available = 't'
),

-- create new_table and float_table to change the price from text to float
new_table AS (
SELECT 
	get_month.date_part,
	REPLACE(get_month.new_price, '$', '') AS new_price2
FROM 
 	get_month
),
float_table AS (
	SELECT 
		date_part,
		CAST (new_price2 AS float)
	FROM
		new_table
)

-- find the avg, min and max prices for listings which have availability as determined by the sfo_calender table.
SELECT
	date_part,
	AVG(new_price2) AS average_price,
	MIN(new_price2) AS min_price,
	MAX(new_price2) AS max_price
FROM 
	float_table
GROUP BY 
	date_part
ORDER BY
	average_price;
~~~~

Output: Columns are month, avg price, min price and max price.
~~~~
"4"	"214.287218577944"	"10"	"22625"
"3"	"218.815850325836"	"10"	"220713"
"5"	"220.897306516322"	"10"	"23694"
"11"	"222.648999419954"	"28"	"24222"
"10"	"224.875831302894"	"28"	"24222"
"1"	"226.754231427219"	"10"	"162032"
"6"	"227.576895619507"	"10"	"24222"
"9"	"229.409688083243"	"28"	"24222"
"12"	"232.323379796287"	"10"	"135985"
"8"	"232.504816165468"	"10"	"24222"
"7"	"232.591136444277"	"10"	"24222"
"2"	"235.360793972514"	"10"	"220713"
~~~~

Answer:  Months of March, April and May are the cheapest months by average, so Spring is the cheapest time to go to San Francisco.  The busiest time of year is presumably at the time where prices are most expensive, so Feb, July and August seem to be the busiest.  It looks like there might be so erroneous data, given there are max values of 220,713.00 for the month of February and March; January also has a max value of 162,032.00.  These high values could be due to the fact that they span many days or weeks or months, so more investigation is needed.  

A better way to evaluate the prices per listing id would be to analyze the history of prices obtained per listing id, so we can see that prices where the renatals actually occurred. This information was not available as part of this exercise.